#1. Data

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print(tf.__version__)
print(hub.__version__)
print('GPU', 'On' if tf.config.experimental.list_physical_devices("GPU") else 'Off')

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]','train[60%:]','test'),
    as_supervised=True
)

In [ ]:
len(train_data), len(validation_data), len(test_data)

In [ ]:
print(train_data)

In [ ]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_example_batch

In [ ]:
train_labels_batch

#2. Model

In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_example_batch[:5])

In [ ]:
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20, validation_data=validation_data.batch(512))

In [ ]:
results = model.evaluate(test_data.batch(512))

for metrics, value in zip(model.metrics_names, results):
    print("Metrics: %s Value: %.3f" % (metrics, value))